In [1]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap
pd.set_option('display.max_columns', None)


In [2]:
df_Berlin = pd.read_csv('../livablestreets/data/Livability_Berlin_grid_1000m.csv')
df_Berlin.head()

,lat_start,lat_end,lng_start,lng_end,lat_center,lng_center,km_to_centroid,grid_in_berlin,degrees_to_centroid,activities_economic,activities_education,activities_health_care,activities_public_service,comfort_leisure_sports,comfort_sports,convenience,mobility_public_transport,social_community,social_culture,social_eating,social_night_life,activities_mean,comfort_mean,mobility_mean,social_mean,livability
0,52.662246,52.671246,13.088348,13.107053,52.666746,13.097701,27.589830,False,151.520290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,52.662246,52.671246,13.107053,13.125758,52.666746,13.116406,26.660446,False,149.972553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52.662246,52.671246,13.125758,13.144463,52.666746,13.135111,25.759544,False,148.265275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.662246,52.671246,13.144463,13.163169,52.666746,13.153816,24.890218,False,146.376003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,52.662246,52.671246,13.163169,13.181874,52.666746,13.172521,24.055891,False,144.278839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Take only the ones inside Berlin

In [3]:
df_Berlin = df_Berlin[df_Berlin['grid_in_berlin']==True]

## Create df for each category

In [4]:
columns = df_Berlin.columns
columns

Index(['lat_start', 'lat_end', 'lng_start', 'lng_end', 'lat_center',
       'lng_center', 'km_to_centroid', 'grid_in_berlin', 'degrees_to_centroid',
       'activities_economic', 'activities_education', 'activities_health_care',
       'activities_public_service', 'comfort_leisure_sports', 'comfort_sports',
       'convenience', 'mobility_public_transport', 'social_community',
       'social_culture', 'social_eating', 'social_night_life',
       'activities_mean', 'comfort_mean', 'mobility_mean', 'social_mean',
       'livability'],
      dtype='object')

### Get all categories that end with _mean and if present 'livability'

In [6]:
def get_category_names(df):
    columns = df.columns
    columns_categories = [col for col in columns if col.split('_')[-1]=='mean']
    if 'livability' in columns:
        columns_categories.append('livability')
    return columns_categories

In [9]:
column_categories = get_category_names(df_Berlin)
column_categories

['activities_mean',
 'comfort_mean',
 'mobility_mean',
 'social_mean',
 'livability']

### Create df for each category returned as a dictionary with {cat_name: df}

In [10]:
def create_category_array(df):
    column_category_mean = get_category_names(df)
    categories={}
    for cat in column_category_mean:
        categories[cat] = np.array(df[['lat_center','lng_center', cat]])
    return categories

In [11]:
categories = create_category_array(df_Berlin)
categories

{'activities_mean': array([[5.26667463e+01, 1.34718039e+01, 4.73988838e-02],
        [5.26577463e+01, 1.32847522e+01, 6.21047661e-02],
        [5.26577463e+01, 1.33034574e+01, 4.73988838e-02],
        ...,
        [5.23607460e+01, 1.36588555e+01, 4.73988838e-02],
        [5.23517460e+01, 1.36401504e+01, 6.52560267e-02],
        [5.23427460e+01, 1.36401504e+01, 4.73988838e-02]]),
 'comfort_mean': array([[52.66674633, 13.47180389,  0.07272727],
        [52.65774632, 13.28475223,  0.09545455],
        [52.65774632, 13.3034574 ,  0.07272727],
        ...,
        [52.36074602, 13.65885554,  0.07272727],
        [52.35174601, 13.64015037,  0.07272727],
        [52.342746  , 13.64015037,  0.07272727]]),
 'mobility_mean': array([[5.26667463e+01, 1.34718039e+01, 5.55555556e-03],
        [5.26577463e+01, 1.32847522e+01, 5.55555556e-03],
        [5.26577463e+01, 1.33034574e+01, 5.55555556e-03],
        ...,
        [5.23607460e+01, 1.36588555e+01, 5.55555556e-03],
        [5.23517460e+01, 1.3640

## Create heatmaps for each feature

In [14]:
def create_heatmap(data:dict):
    heatmaps={}
    for category in data.keys():
        heatmaps[category]=HeatMap(data[category], 
                          min_opacity=0.2,
                          gradient={0:'Navy', 0.25:'Blue',0.5:'Green', 0.75:'Yellow',1: 'Red'},
                          radius=20, 
                          name=category, 
                          show=False)
    return heatmaps

In [15]:
heatmaps = create_heatmap(categories)

In [17]:
def plot_heatmaps(heatmaps):
    mapObj = folium.Map(location=[52.5200, 13.4050], zoom_start=10) #hardcoded for Berlin
    for hm in heatmaps.values():
        mapObj.add_child(hm)
    folium.LayerControl().add_to(mapObj)
    return mapObj

In [18]:
plot_heatmaps(heatmaps)